In [60]:
import dotenv, os
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
spotify_REDIRECT_URI = os.getenv('SPOTIFY_REDIRECT_URI')

In [61]:
import requests

def get_spotify_auth_url():
    return 'https://accounts.spotify.com/authorize?client_id={}&response_type=code&redirect_uri={}&scope=user-read-private%20user-read-email%20user-read-playback-state%20user-modify-playback-state&state=34fFs29kd09'.format(SPOTIFY_CLIENT_ID, spotify_REDIRECT_URI)

In [42]:
import flask
import json

app = flask.Flask(__name__)

oauth_token = None

@app.route('/login')
def login(): 
    return flask.redirect(get_spotify_auth_url())

@app.route('/callback/spotify')
def spotify_callback():
    code = flask.request.args.get('code')
    data = {
        'grant_type': 'authorization_code',
        'code': code,
        'redirect_uri': spotify_REDIRECT_URI,
        'client_id': SPOTIFY_CLIENT_ID,
        'client_secret': SPOTIFY_CLIENT_SECRET
    }
    response = requests.post('https://accounts.spotify.com/api/token', data=data)
    global oauth_token
    oauth_token = response.json()['access_token']
    return flask.redirect('/')

@app.route('/playlist/<playlist_id>')
def playlist(playlist_id):
    next = 'https://api.spotify.com/v1/playlists/{}/tracks'.format(playlist_id)
    all_items = []
    while True:
        print(next)
        response = requests.get(next, headers={'Authorization': 'Bearer {}'.format(oauth_token)})
        data = response.json()
        if not 'items' in data: break
        all_items += data['items']
        if not 'next' in data: break
        next = data['next']
        if not next: break
    with open('playlist.json', 'w') as f:
        f.write(json.dumps(all_items, indent=4))
    return all_items

app.run(port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [05/Feb/2024 14:08:19] "GET /login HTTP/1.1" 302 -
127.0.0.1 - - [05/Feb/2024 14:08:19] "GET /login HTTP/1.1" 302 -
127.0.0.1 - - [05/Feb/2024 14:08:20] "GET /callback/spotify?code=AQD79Q3CVtor5CmGmWzR6cQxIrFLnZTBso2aQ3LohR3EQZ3zXW99Zr4CyjJep5iZ4F5JaWlMNm5c2jmbH4HeAONGtM6erJZL8z5nxzT6kIlIczjc42OY29h4VdMBgjT53KXl3gi7jWDADWh8A4knTkbh1dS6RhU4DiJ8Xfl7nyFIZZbSfL3hBq4qF6riJqOO9zKKhQ0YRLBT1rhprUtyzpLJvLTAmxB1Ti30Z8rgypsCOHbYZjDOa9YCHuz9UaZrpYakjd75C1sitFGVv3ofxBmoPL5P416v2Z124v1YTjh0BZxf&state=34fFs29kd09 HTTP/1.1" 302 -
127.0.0.1 - - [05/Feb/2024 14:08:20] "GET / HTTP/1.1" 404 -


https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=100&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=200&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=300&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=400&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=500&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=600&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=700&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=800&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=900&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyFbjsH1Mh/tracks?offset=1000&limit=100
https://api.spotify.com/v1/playlists/1DTzz7Nh2rJBnyF

127.0.0.1 - - [05/Feb/2024 14:08:37] "GET /playlist/1DTzz7Nh2rJBnyFbjsH1Mh HTTP/1.1" 200 -


In [49]:
with open('playlist.json') as f:
    playlist_tracks = json.load(f)

print(oauth_token)

BQCwWZ-HM8REB5oI6L_ekCk6fFz8hunp6SpRn7Bz90zfgupq-BvUcdqc4dZfAR4lcmGE5tWnok3DXUHabg532zDOlbIVAEM-kNzOEOdUnJPbcxDr7b211i8ko4xSAeYF96v4C8V_2XJ0DP35sB3Urx9stcEjxFrO8raCCNUpOA2DSYCzY2dq2WuOscf_sQ6rpO1WN2X-


In [50]:
def getTrackFeatures(ids):
    return requests.get('https://api.spotify.com/v1/audio-features?ids={}'.format(ids), headers={
        'Authorization': 'Bearer {}'.format(oauth_token)
    }).json()['audio_features']
    

for i in range(0, len(playlist_tracks), 100):
    track_ids = [track['track']['id'] for track in playlist_tracks[i:i+100]]
    features = getTrackFeatures(",".join(track_ids))
    print(features)
    for feature in features:
        print(feature)
        id = feature['id']
        for track in playlist_tracks:
            if track['track']['id'] == id:
                track['features'] = feature
                break

with open('playlist_with_features.json', 'w') as f:
    f.write(json.dumps(playlist_tracks, indent=4))

[{'danceability': 0.39, 'energy': 0.397, 'key': 0, 'loudness': -9.963, 'mode': 0, 'speechiness': 0.0513, 'acousticness': 0.287, 'instrumentalness': 0, 'liveness': 0.207, 'valence': 0.246, 'tempo': 144.031, 'type': 'audio_features', 'id': '3z8h0TU7ReDPLIbEnYhWZb', 'uri': 'spotify:track:3z8h0TU7ReDPLIbEnYhWZb', 'track_href': 'https://api.spotify.com/v1/tracks/3z8h0TU7ReDPLIbEnYhWZb', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3z8h0TU7ReDPLIbEnYhWZb', 'duration_ms': 354947, 'time_signature': 4}, {'danceability': 0.401, 'energy': 0.383, 'key': 9, 'loudness': -10.048, 'mode': 1, 'speechiness': 0.0279, 'acousticness': 0.51, 'instrumentalness': 0.0078, 'liveness': 0.121, 'valence': 0.285, 'tempo': 96.957, 'type': 'audio_features', 'id': '5B5YKjgne3TZzNpMsN9aj1', 'uri': 'spotify:track:5B5YKjgne3TZzNpMsN9aj1', 'track_href': 'https://api.spotify.com/v1/tracks/5B5YKjgne3TZzNpMsN9aj1', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5B5YKjgne3TZzNpMsN9aj1', 'duration_ms'

In [65]:

new_playlist_tracks = []

for index, item in enumerate(playlist_tracks):
    if 'features' in item:
        obj = {
            'position': index+1,
            'id': item['track']['id'],
            'name': item['track']['name'],
            'artist': item['track']['artists'][0]['name']
        }
        for key in item['features']: obj[key] = item['features'][key]
        new_playlist_tracks.append(obj)
import csv
with open('playlist.csv', 'w', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=new_playlist_tracks[0].keys())
    writer.writeheader()
    for row in new_playlist_tracks:
        writer.writerow(row)

In [59]:
new_playlist_tracks

[{'position': 1,
  'id': '3z8h0TU7ReDPLIbEnYhWZb',
  'name': 'Bohemian Rhapsody',
  'artist': 'Queen',
  'danceability': 0.39,
  'energy': 0.397,
  'key': 0,
  'loudness': -9.963,
  'mode': 0,
  'speechiness': 0.0513,
  'acousticness': 0.287,
  'instrumentalness': 0,
  'liveness': 0.207,
  'valence': 0.246,
  'tempo': 144.031,
  'type': 'audio_features',
  'uri': 'spotify:track:3z8h0TU7ReDPLIbEnYhWZb',
  'track_href': 'https://api.spotify.com/v1/tracks/3z8h0TU7ReDPLIbEnYhWZb',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3z8h0TU7ReDPLIbEnYhWZb',
  'duration_ms': 354947,
  'time_signature': 4},
 {'position': 2,
  'id': '5B5YKjgne3TZzNpMsN9aj1',
  'name': 'Roller Coaster',
  'artist': 'Danny Vera',
  'danceability': 0.401,
  'energy': 0.383,
  'key': 9,
  'loudness': -10.048,
  'mode': 1,
  'speechiness': 0.0279,
  'acousticness': 0.51,
  'instrumentalness': 0.0078,
  'liveness': 0.121,
  'valence': 0.285,
  'tempo': 96.957,
  'type': 'audio_features',
  'uri': 'spotify:t